# Interactions - Lab

## Introduction

In this lab, you'll explore interactions in the Boston Housing data set.

## Objectives

You will be able to:
- Understand what interactions are
- Understand how to accommodate for interactions in regression

## Build a baseline model 

You'll use a couple of built-in functions, which we imported for you below.

In [1]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.datasets import load_boston
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Import the Boston data set using `load_boston()`. We won't bother to preprocess the data in this lab. If you still want to build a model in the end, you can do that, but this lab will just focus on finding meaningful insights in interactions and how they can improve $R^2$ values.

In [2]:
regression = LinearRegression()
boston = load_boston()

Create a baseline model which includes all the variables in the Boston housing data set to predict the house prices. Then use 10-fold cross-validation and report the mean $R^2$ value as the baseline $R^2$.

In [40]:
## code here
X = pd.DataFrame(boston['data'])
X.columns = boston['feature_names']
y = pd.DataFrame(boston['target'])
y.columns = ['MEDV']
all_data = pd.concat([X, y], axis=1)            
    
crossvalidation = KFold(n_splits=10, shuffle=True, random_state=1)
baseline = np.mean(cross_val_score(regression, X, y, scoring="r2", cv=crossvalidation))

In [17]:
baseline

0.7190106820189475

## See how interactions improve your baseline

Next, create all possible combinations of interactions, loop over them and add them to the baseline model one by one to see how they affect the $R^2$. We'll look at the 3 interactions which have the biggest effect on our $R^2$, so print out the top 3 combinations.

You will create a for loop to loop through all the combinations of 2 predictors. You can use `combinations` from itertools to create a list of all the pairwise combinations. To find more info on how this is done, have a look [here](https://docs.python.org/2/library/itertools.html).

In [18]:
from itertools import combinations
combinations = list(combinations(boston.feature_names, 2))

In [20]:
combinations[:5]

[('CRIM', 'ZN'),
 ('CRIM', 'INDUS'),
 ('CRIM', 'CHAS'),
 ('CRIM', 'NOX'),
 ('CRIM', 'RM')]

In [22]:
combinations[0]

('CRIM', 'ZN')

In [24]:
combinations[0][0]+"_"+combinations[0][1]

'CRIM_ZN'

In [27]:
## code to find top 3 interactions by R^2 value here
interation_R = []
for combination in combinations:
    X_interact = X.copy()
    X_interact[combination[0]+"_"+combination[1]] = X[combination[0]] * X[combination[1]]
    interact_R = np.mean(cross_val_score(regression,
                                                    X_interact,
                                                    y,
                                                    scoring="r2",
                                                    cv=crossvalidation))
    interation_R.append([combination, interact_R])

In [33]:
interation_R.sort(key=lambda x: x[1], reverse=True)
interation_R

[[('RM', 'LSTAT'), 0.7864889421124032],
 [('RM', 'TAX'), 0.7750525123747651],
 [('RM', 'RAD'), 0.7682152400234054],
 [('RM', 'PTRATIO'), 0.7633759067582868],
 [('INDUS', 'RM'), 0.7577805143520179],
 [('NOX', 'RM'), 0.7453599495065311],
 [('RM', 'AGE'), 0.7406598760890253],
 [('RM', 'B'), 0.7385665707082879],
 [('RM', 'DIS'), 0.7342130965590978],
 [('ZN', 'RM'), 0.72882064115271],
 [('INDUS', 'DIS'), 0.7283635108497113],
 [('ZN', 'INDUS'), 0.7245130249509673],
 [('CRIM', 'CHAS'), 0.7224266827041406],
 [('INDUS', 'TAX'), 0.7223792494260635],
 [('INDUS', 'AGE'), 0.7222863283272001],
 [('TAX', 'PTRATIO'), 0.7220753714337268],
 [('CRIM', 'RAD'), 0.7218426607693014],
 [('NOX', 'RAD'), 0.7216402620446948],
 [('DIS', 'PTRATIO'), 0.7212530527937477],
 [('CRIM', 'TAX'), 0.721193712503856],
 [('AGE', 'DIS'), 0.7210193945912328],
 [('CHAS', 'LSTAT'), 0.7210130989247112],
 [('CRIM', 'NOX'), 0.7209568446771786],
 [('AGE', 'B'), 0.7209442240026325],
 [('NOX', 'PTRATIO'), 0.7208556256614026],
 [('CRIM

## Look at the top 3 interactions: "RM" as a confounding factor

The top three interactions seem to involve "RM", the number of rooms as a confounding variable for all of them. Let's have a look at interaction plots for all three of them. This exercise will involve:

- splitting the data up in 3 groups: one for houses with a few rooms, one for houses with a "medium" amount of rooms, one for a high amount of rooms.
- Create a function `build_interaction_rm`. This function takes an argument `varname` (which can be set equal to the column name as a string) and a column `description` (which describes the variable or varname, to be included on the x-axis of the plot). The function outputs a plot that uses "RM" as a confounding factor. Each plot should have three regression lines, one for each level of "RM." 

The data has been split into high, medium and low number of rooms for you.

In [36]:
rm = np.asarray(X[["RM"]]).reshape(len(X[["RM"]]))

In [43]:
rm

array([6.575, 6.421, 7.185, 6.998, 7.147, 6.43 , 6.012, 6.172, 5.631,
       6.004, 6.377, 6.009, 5.889, 5.949, 6.096, 5.834, 5.935, 5.99 ,
       5.456, 5.727, 5.57 , 5.965, 6.142, 5.813, 5.924, 5.599, 5.813,
       6.047, 6.495, 6.674, 5.713, 6.072, 5.95 , 5.701, 6.096, 5.933,
       5.841, 5.85 , 5.966, 6.595, 7.024, 6.77 , 6.169, 6.211, 6.069,
       5.682, 5.786, 6.03 , 5.399, 5.602, 5.963, 6.115, 6.511, 5.998,
       5.888, 7.249, 6.383, 6.816, 6.145, 5.927, 5.741, 5.966, 6.456,
       6.762, 7.104, 6.29 , 5.787, 5.878, 5.594, 5.885, 6.417, 5.961,
       6.065, 6.245, 6.273, 6.286, 6.279, 6.14 , 6.232, 5.874, 6.727,
       6.619, 6.302, 6.167, 6.389, 6.63 , 6.015, 6.121, 7.007, 7.079,
       6.417, 6.405, 6.442, 6.211, 6.249, 6.625, 6.163, 8.069, 7.82 ,
       7.416, 6.727, 6.781, 6.405, 6.137, 6.167, 5.851, 5.836, 6.127,
       6.474, 6.229, 6.195, 6.715, 5.913, 6.092, 6.254, 5.928, 6.176,
       6.021, 5.872, 5.731, 5.87 , 6.004, 5.961, 5.856, 5.879, 5.986,
       5.613, 5.693,

In [42]:
high_rm = all_data[rm > np.percentile(rm, 67)]
med_rm = all_data[(rm > np.percentile(rm, 33)) & (rm <= np.percentile(rm, 67))]
low_rm = all_data[rm <= np.percentile(rm, 33)]

Create `build_interaction_rm`.

In [ ]:
def build_interaction_rm(varname, description):
    plt.figure(figsize=(10,6))
    plt.scatter()

Next, use build_interaction_rm with the three variables that came out with the highest effect on $R^2$

In [ ]:
# first plot

In [ ]:
# second plot

In [ ]:
# third plot

## Build a final model including all three interactions at once

Use 10-fold cross validation.

In [ ]:
# code here

In [ ]:
# code here

Our $R^2$ has increased considerably! Let's have a look in statsmodels to see if all these interactions are significant.

In [ ]:
# code here

What is your conclusion here?

In [ ]:
# formulate your conclusion

## Summary

You should now understand how to include interaction effects in your model! As you can see, interactions can have a strong impact on linear regression models, and they should always be considered when you are constructing your models.